## Hämta kyrkogårdar FindAGrave

* [GITHUB](https://github.com/salgo60/Gravstensinventeringen-Wikidata)
* Wikidata 
  * Find-A-Grave begravningsplatsid [Property P2025](https://www.wikidata.org/wiki/Property:P2025)
      * SPARQL [kyrkogårdar Sverige](https://w.wiki/65LZ)
      * SPARQL [alla kopplade objekt med koordinat](https://w.wiki/65C3)
      * SPARQL [alla kopplade objekt med koordinat och en:Wikipediartikel](https://w.wiki/65C3)
  * Gravstensinventeringen [Property:P5259](https://www.wikidata.org/wiki/Property:P5259)


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-12-10 13:47:18.328141


In [2]:
class Gravplats: 
    def __init__(self, nameCemetery, urlCemetery, findaGraveID, wd, county, kommun):
        self.nameCemetery = nameCemetery
        self.urlCemetery = urlCemetery
        self.findaGraveID = findaGraveID
        self.Wikidata = wd
        self.county = county
        self.kommun = kommun        
        
    def __repr__(self):
        return f"Grav: ({self.nameCemetery!r}, {self.urlCemetery!r}, {self.findaGraveID!r}, \
                        {self.Wikidata!r}, {self.county!r}, {self.kommun!r}) "

    def __iter__(self):
        return iter([self.nameCemetery, self.urlCemetery, self.findaGraveID, self.Wikidata,
                    self.county, self.kommun])


In [3]:
import json
def getWD (id):
    # https://hub.toolforge.org/P2025:1972857?format=json
    baseurl = "https://hub.toolforge.org/P2025:"
    huburl = baseurl + str(id) + "?format=json"
    responseHub = requests.get(huburl)
    hubJson = json.loads(responseHub.text)

    wd = ""
    if "origin" in hubJson:
        wd = hubJson["origin"]["qid"]
    
    return wd

In [ ]:
import requests
import urllib.request
import time,re
from tqdm import tqdm
from bs4 import BeautifulSoup as beauty
import cloudscraper

urlCountry = "https://www.findagrave.com/cemetery-browse/Sweden?id=country_66"
scraper = cloudscraper.create_scraper(delay=10, browser='chrome') 

listCemeteries = []

try:
    info = scraper.get(urlCountry).text
    soup = beauty(info, "html.parser")
    s = soup.find_all('a')
    for a in s:
        try:
            href = a["href"]
            if "Sweden" in href:
                countyString = href.split("/")[3].split("?")[0].replace("-"," ")
                print(countyString, " - ", href)
                urlCounty = "https://www.findagrave.com/" + href
                #print("\t", urlCounty)
                info = scraper.get(urlCounty).text
                soupCounty = beauty(info, "html.parser")
                sCounty = soupCounty.find_all('a')
                for m in sCounty:
                    try: 
                        hrefC = m["href"]
                        #print(m.hrefC,hrefC)
                        if "county_" in hrefC:
                            kommunString = hrefC.split("/")[4].split("?")[0].replace("-"," ")
                            urlM = "https://www.findagrave.com" + hrefC
                            print("\t",kommunString, urlM)
                            infoM = scraper.get(urlM).text
                            soupM = beauty(infoM, "html.parser")
                            sM = soupM.find_all('a')
                            for c in sM:
                                hrefMc = c["href"]
                                if "id=city_" in hrefMc:
                                    
                                    cityUrl = hrefMc
                                    urlMc = "https://www.findagrave.com" + hrefMc
                                    #print ("urlMc ",urlMc)
                                    infoMc = scraper.get(urlMc).text
                                    soupMc = beauty(infoMc, "html.parser")
                                    soupMc_a = soupMc.find_all('a')
                                    for aMc in soupMc_a:
                                        hrefGr = aMc["href"]
                                        if "/cemetery/" in hrefGr:
                                            #print ("hrefGr ",hrefGr)
                                            urlGr = "https://www.findagrave.com" + hrefGr
                                            infoGr = scraper.get(urlGr).text
                                            #print ("infoGr ",infoGr)
                                            soupGr = beauty(infoGr, "html.parser")
                                            Grclassname="bio-name m-0"
                                            Grname_h1 = soupGr.find_all("h1", {"class": Grclassname})[0]
                                            #print ("Grname_h1 ",Grname_h1,hrefGr)
                                            FindAgraveid = hrefGr.split("/")[2]
                                            CemeteryName = Grname_h1.text.strip()
                                            
                                            wdobj = getWD(FindAgraveid)
                                            if not wdobj:
                                                print(urlGr,CemeteryName,FindAgraveid,wdobj,countyString, kommunString)
                                                gravObj =  Gravplats(CemeteryName,
                                                            urlGr,
                                                            FindAgraveid,
                                                            wdobj,
                                                            countyString,
                                                            kommunString)
                                                listCemeteries.append(gravObj)
                                            else:
                                                print ("WDobj")
                                    print ("\t\t\t",len(listCemeteries) ," antal poster listCemeteries")
                            print ("\t",len(listCemeteries) ," antal poster listCemeteries")                  
                                    
                                
                    except:
                        pass
        except:
            print("\t\t error ",a)
                        
except :
    print("error " + url)
    time.sleep(1)
    
print (len(listCemeteries) ," antal poster")

In [5]:
#listCemeteries


In [6]:
listCemeteries

[Grav: ('Aaryd Kyrkogård', 'https://www.findagrave.com/cemetery/2573569/aaryd-kyrkog%C3%A5rd', '2573569',                         'Q115301664', 'Blekinge län', 'Karlshamns kommun') ,
 Grav: ('Aspö kyrkogård', 'https://www.findagrave.com/cemetery/2709544/asp%C3%B6-kyrkog%C3%A5rd', '2709544',                         'Q115301673', 'Blekinge län', 'Karlskrona kommun') ,
 Grav: ('Avesta kyrkogård', 'https://www.findagrave.com/cemetery/2640334/avesta-kyrkog%C3%A5rd', '2640334',                         'Q29555937', 'Dalarnas län', 'Avesta kommun') ,
 Grav: ('Envikens Kyrkogård', 'https://www.findagrave.com/cemetery/2590829/envikens-kyrkog%C3%A5rd', '2590829',                         '', 'Dalarnas län', 'Falu kommun') ,
 Grav: ('Gagnefs kyrkogård', 'https://www.findagrave.com/cemetery/2546338/gagnefs-kyrkog%C3%A5rd', '2546338',                         '', 'Dalarnas län', 'Gagnefs kommun') ,
 Grav: ('Stjarnsundkyrka', 'https://www.findagrave.com/cemetery/2401910/stjarnsundkyrka', '2401910',    

In [18]:
import pandas as pd
df = pd.DataFrame(listCemeteries)
df.columns = ["Namn","url","FindaGrave","Wikidata","Län","Kommun"]
#df.to_csv("FindAGrave.csv")

In [19]:
df

,Namn,url,FindaGrave,Wikidata,Län,Kommun
0,Aaryd Kyrkogård,https://www.findagrave.com/cemetery/2573569/aa...,2573569,Q115301664,Blekinge län,Karlshamns kommun
1,Aspö kyrkogård,https://www.findagrave.com/cemetery/2709544/as...,2709544,Q115301673,Blekinge län,Karlskrona kommun
2,Avesta kyrkogård,https://www.findagrave.com/cemetery/2640334/av...,2640334,Q29555937,Dalarnas län,Avesta kommun
3,Envikens Kyrkogård,https://www.findagrave.com/cemetery/2590829/en...,2590829,,Dalarnas län,Falu kommun
4,Gagnefs kyrkogård,https://www.findagrave.com/cemetery/2546338/ga...,2546338,,Dalarnas län,Gagnefs kommun
...,...,...,...,...,...,...
155,Ask Kyrkogard,https://www.findagrave.com/cemetery/2492267/as...,2492267,,Östergötlands län,Motala kommun
156,Dagsberg,https://www.findagrave.com/cemetery/2579032/da...,2579032,,Östergötlands län,Norrköpings kommun
157,Rogslösa Kyrkogård,https://www.findagrave.com/cemetery/2729277/ro...,2729277,Q115304941,Östergötlands län,Vadstena kommun
158,Kyrkogård Ringarum,https://www.findagrave.com/cemetery/2545850/ky...,2545850,,Östergötlands län,Valdemarsviks kommun


In [8]:
end = datetime.now()
print("Ended: ", end)
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))


Ended:  2022-12-10 13:56:17.302688
Time elapsed (hh:mm:ss.ms) 0:08:58.974735
